针对 data/路径规划_数据.csv ，调用API，获取数据即可

json 文件保存，字典的键使用 ((),())

In [23]:
import os
import sys
import pandas as pd
import asyncio
from aiolimiter import AsyncLimiter
from tqdm import tqdm
import aiohttp
import json
import pandas as pd

from utils import save_json, load_json
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv()
AK = os.getenv("api_key")

## API 调用示例

In [24]:
df = pd.read_csv("data/路径规划_数据.csv")

In [25]:
df.head(2)

,origin_prov,origin_city,origin_经度,origin_纬度,target_prov,target_city,target_经度,target_纬度
0,辽宁省,辽阳市,123.237,41.267800,陕西省,榆林市,109.735,38.285201
1,吉林省,吉林市,126.549,43.837899,四川省,攀枝花市,101.719,26.582199


In [26]:
# BASE_URL = "https://api.map.baidu.com/direction/v2/driving"
BASE_URL = "https://api.map.baidu.com/directionlite/v1/driving"


def get_params(
    origin_lat,
    origin_lon,
    target_lat,
    target_lon,
):
    lon_lat_template = "{:.6f},{:.6f}"
    params = {
        "origin": lon_lat_template.format(origin_lat, origin_lon),
        "destination": lon_lat_template.format(target_lat, target_lon),
        "ak": AK,
    }
    return params

In [27]:
import requests

In [28]:
df.shape[0]

56616

In [29]:
row.keys()

Index(['origin_prov', 'origin_city', 'origin_经度', 'origin_纬度', 'target_prov',
       'target_city', 'target_经度', 'target_纬度'],
      dtype='object')

In [31]:
for _, row in df.iterrows():
    origin_lon = row["origin_经度"]
    origin_lat = row["origin_纬度"]
    target_lon = row["target_经度"]
    target_lat = row["target_纬度"]
    params = get_params(
        origin_lat=origin_lat,
        origin_lon=origin_lon,
        target_lat=target_lat,
        target_lon=target_lon,
    )

    address_key = (
        (row["origin_prov"], row["origin_city"]),
        (
            row["target_prov"],
            row["target_city"],
        ),
    )
    response = requests.get(BASE_URL, params=params)

    break

In [32]:
response.status_code

200

In [33]:
df.head(2)

,origin_prov,origin_city,origin_经度,origin_纬度,target_prov,target_city,target_经度,target_纬度
0,辽宁省,辽阳市,123.237,41.267800,陕西省,榆林市,109.735,38.285201
1,吉林省,吉林市,126.549,43.837899,四川省,攀枝花市,101.719,26.582199


In [22]:
address_key

(('辽宁省', '辽阳市'), ('陕西省', '榆林市'))

In [7]:
response.text

'{"status":0,"message":"ok","result":{"origin":{"lng":123.237,"lat":41.26779992744},"destination":{"lng":109.735,"lat":38.285201067552},"routes":[{"route_md5":"3c68a39f8c1b7c14e09b3a91fb1c0423","distance":1406144,"duration":57349,"traffic_condition":1,"toll":653,"restriction_info":{"status":0},"steps":[{"leg_index":0,"distance":133,"duration":9,"direction":10,"turn":7,"road_type":3,"road_types":"3","instruction":"\\u4ece<b>\\u8d77\\u70b9<\\/b>\\u5411\\u897f\\u5317\\u65b9\\u5411\\u51fa\\u53d1,\\u6cbf<b>\\u5929\\u798f\\u8def<\\/b>\\u884c\\u9a76130\\u7c73,<b>\\u5de6\\u8f6c<\\/b>\\u8fdb\\u5165<b>\\u8944\\u5e73\\u5927\\u8857<\\/b>","path":"123.23677562882,41.267408571186;123.23566963507,41.268027739831;123.23559992657,41.268077083284;123.23552015704,41.268026248682","traffic_condition":[{"status":1,"geo_cnt":3}],"start_location":{"lng":"123.23677562882","lat":"41.267408571186"},"end_location":{"lng":"123.23552015704","lat":"41.268026248682"}},{"leg_index":0,"distance":1089,"duration":43,"di

In [10]:
type(response.json())

dict

In [15]:
len(response.json()["result"]["routes"])

1

In [17]:
response.json()["result"]["routes"][0]["distance"], response.json()["result"]["routes"][0]["duration"]

(1406144, 57349)